In [ ]:
from lark import Lark, tree
from lark.visitors import CollapseAmbiguities
import matplotlib.pyplot as plt

In [ ]:
def plot_trees(grammar:str,  text:str, start='expr'):
    parser = Lark(grammar=grammar, start=start,ambiguity='explicit')  
    parsed = parser.parse(text)
    trees = CollapseAmbiguities().transform(parsed)
    for i, t in enumerate(trees):
        tree.pydot__tree_to_png(t, filename=f'tree{i}.png', rankdir='TB')
        plt.figure(figsize=(8,8))
        plt.imshow(plt.imread(f"tree{i}.png"))
        plt.show()

Defining grammar in EBNF notation is fairly simple except one thing - operators priority and associativity. There are a lot of questions in the internet about it. That's why I'm going explain step-by-step how to set operator precendence and associativity for a simple grammar.

Motivating example

(all examples are written using lark library)

Cosider we would like to create a grammar for basic arithmetic operations like 1+2+3, 7/8+9-2. Later we will expand it to support unary operators (eg: -5), exponentiation (eg: 2^3) and parentheses (eg: (1+2)*3).

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

BINARY_OP: "+" | "-" | "*" | "/"
!expr : expr BINARY_OP expr
    | NUMBER

%ignore WS
"""

The grammar is very simple, we have four operators that can work on two numbers. The expression can be either a single number or a sequence of binary operations. 

In [ ]:
plot_trees(grammar, '9/3')

But there is a problem. Consider a very simple case: 1+2+3. In fact, our grammar is ambigous and can produce two different results!

In [ ]:
plot_trees(grammar, '1+2+2')

First one is equivalent to (1+2)+3=3+3=6, the second one is equivalent to 1+(2+3)=3+3=6. For addition it is not a problem as the result is the same. But, for substraction we have:

In [ ]:
plot_trees(grammar, '1-2-3')

The (1-2)-3=-1-3=-4 is not the same as 1-(2-3)=1-(-1)=0!

The reason is associativity. For additon a+b+c=(a+b)+c=a+(b+c) but it not the case for substraction. Our current grammar does not define operators associativty so it can be interpreted eiter way. Fortunately, all four operations are left-associative (meaning (a⊗b)⊗c not a⊗(b⊗c) where ⊗ denotes any operator).

The fix is simple:

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

BINARY_OP: "+" | "-" | "*" | "/"
!expr : expr BINARY_OP NUMBER
    | NUMBER

%ignore WS
"""
plot_trees(grammar, '1-2-3')

We control left/right associativity by selecting at wchich side of the operator recursion is performed. If we turn it another way around, operators would be right-associative.

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

BINARY_OP: "+" | "-" | "*" | "/"
!expr : NUMBER BINARY_OP expr
    | NUMBER

%ignore WS
"""
plot_trees(grammar, '1-2-3')

Priority/precendence

We solved the first issue, but still priorities are not correct. For example

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

BINARY_OP: "+" | "-" | "*" | "/"
!expr : expr BINARY_OP NUMBER
    | NUMBER

%ignore WS
"""
plot_trees(grammar, '1+2*3')

Grammar is unambigous but incorrect. Let's try to fix it.

For first we need to select how many priority levels we have. For now there are two: */ before +-. For each level we intoduce a separate expression type. As */ needs to be computed before +- we conclude that multiplicative expression should be "under" additive. Let's see an example:

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

ADD_OP: "+" | "-" 
MUL_OP: "*" | "/"

!add_expr: add_expr ADD_OP mul_expr
    | mul_expr

!mul_expr: mul_expr MUL_OP NUMBER
    | NUMBER

!expr: add_expr

%ignore WS
"""
plot_trees(grammar, '1+2*3')

Starting from the bottom:
(me and ae are multiplicative and additive expressions)

Multiplicative expression is either
1. me */ number
2. number

It is the same as before - we only limited allowed operators to */.

Now, the ad should be computed onto result or mp
Additive expression is either:
1. ae +- me - recursion at the left (for left-assoc) and mp at the right (because we have already computed me)
2. me - in case there are no +- operators

The expression now is equivalent to additive expression.

You may think it is not going to work for all cases. But it is. Let's see some examples:


In [ ]:
plot_trees(grammar, '1+2-3')

In [ ]:
plot_trees(grammar, '1/2+3/4')

In [ ]:
plot_trees(grammar, '1+2*3+4')

Exponentiation

Now let's apply our knowledge to itroduce exponentiation operator ^. It will be right-associative and has the highest priority.

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

ADD_OP: "+" | "-" 
MUL_OP: "*" | "/"
POWER_OP: "^"

!add_expr: add_expr ADD_OP mul_expr
    | mul_expr

!mul_expr: mul_expr MUL_OP pow_expr
    | pow_expr

!pow_expr: NUMBER POWER_OP pow_expr
    | NUMBER

!expr: add_expr

%ignore WS
"""

I hope you see the pattern here :) Let's try this.

In [ ]:
plot_trees(grammar, '1+2^3^4*5')

Unary operators

It is time to add unary operators. Let's say we support only unary - for expressions like -2+3 or 7*-5.

In order to do this, we introduce a new non-terminal: primary. Primary can be either a number or a unary operator followed by number.

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

ADD_OP: "+" | "-" 
MUL_OP: "*" | "/"
POWER_OP: "^"

!add_expr: add_expr ADD_OP mul_expr
    | mul_expr

!mul_expr: mul_expr MUL_OP pow_expr
    | pow_expr

!pow_expr: primary POWER_OP pow_expr
    | primary

!primary: "-" NUMBER
    | NUMBER

!expr: add_expr

%ignore WS
"""
# plot_trees(grammar, '-2+3')
plot_trees(grammar, '7*-5')

Parenthesis

The last element is parenthesis support. By definition, an expression in parenthesis has the highest priority. It turns out that the implementation is trivial: all we have to do extend the definition of primary to include parenthessed expression.

In [ ]:
grammar = r"""
%import common.NUMBER
%import common.WS

ADD_OP: "+" | "-" 
MUL_OP: "*" | "/"
POWER_OP: "^"

!add_expr: add_expr ADD_OP mul_expr
    | mul_expr

!mul_expr: mul_expr MUL_OP pow_expr
    | pow_expr

!pow_expr: primary POWER_OP pow_expr
    | primary

!primary: "-" NUMBER
    | NUMBER
    | "(" expr ")"

!expr: add_expr

%ignore WS
"""
# plot_trees(grammar, '2*(3-1)')
plot_trees(grammar, '((-2)+7)-(3/(4+(-1)))')

Closing

We covered how to correctly define EBNF grammar for binary expressions with priorities and associativity. We can easily add additional levels of precedence (programming lanugages typically have about 15 of them) by following the same rules.